In [1]:
%matplotlib inline
import mne
import numpy as np

# Mention the file path to the dataset
filename = 'BCI_Data/A01T.gdf'

raw = mne.io.read_raw_gdf(filename)

print(raw.info)
print(raw.ch_names)


Extracting EDF parameters from C:\codebase\mi_debug\BCI_Data\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...


C:\ProgramData\Anaconda3\lib\site-packages\mne\io\edf\edf.py:1131: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
C:\ProgramData\Anaconda3\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


<Info | 7 non-empty values
 bads: []
 ch_names: EEG-Fz, EEG-0, EEG-1, EEG-2, EEG-3, EEG-4, EEG-5, EEG-C3, EEG-6, ...
 chs: 25 EEG
 custom_ref_applied: False
 highpass: 0.5 Hz
 lowpass: 100.0 Hz
 meas_date: 2005-01-17 12:00:00 UTC
 nchan: 25
 projs: []
 sfreq: 250.0 Hz
>
['EEG-Fz', 'EEG-0', 'EEG-1', 'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-Cz', 'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-9', 'EEG-10', 'EEG-11', 'EEG-12', 'EEG-13', 'EEG-14', 'EEG-Pz', 'EEG-15', 'EEG-16', 'EOG-left', 'EOG-central', 'EOG-right']


In [2]:
events, _ = mne.events_from_annotations(raw)

raw.load_data()

raw.filter(7., 35., fir_design='firwin')

raw.info['bads'] += ['EOG-left', 'EOG-central', 'EOG-right']

picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False,
                       exclude='bads')

tmin, tmax = 1., 4.

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 413 samples (1.652 sec)



In [3]:
event_id = dict({'769': 7,'770': 8,'771': 9,'772': 10})
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True)

Not setting metadata
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Loading data for 288 events and 751 original time points ...
0 bad epochs dropped


In [4]:
# Getting labels and changing labels from 7,8,9,10 -> 1,2,3,4
y = epochs.events[:,-1] - 7 

X = epochs.get_data()



In [5]:
X.shape

(288, 22, 751)

In [ ]:
import numpy as np
import datetime
# mne imports
import mne
from mne import io
from mne.datasets import sample

# EEGNet-specific imports
from Algorithm.EEGModels import *
import tensorflow as tf
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

# PyRiemann imports

# tools for plotting confusion matrices
from matplotlib import pyplot as plt

kernels, chans, samples = 1, 22, 751

X_train      = X[0:144,]
Y_train      = y[0:144]
X_validate   = X[144:216,]
Y_validate   = y[144:216]
X_test       = X[216:,]
Y_test       = y[216:]

Y_train      = np_utils.to_categorical(Y_train-1)
Y_validate   = np_utils.to_categorical(Y_validate-1)
Y_test       = np_utils.to_categorical(Y_test-1)

X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
X_validate   = X_validate.reshape(X_validate.shape[0], chans, samples, kernels)
X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)
   
model = EEGNet(nb_classes = 4, Chans = chans, Samples = samples,
               dropoutRate = 0.25)

# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])

fittedModel = model.fit(X_train, Y_train, batch_size = 16,epochs = 4000,
                        verbose = 2, validation_data=(X_validate, Y_validate))

probs       = model.predict(X_test)
preds       = probs.argmax(axis = -1)  
acc         = np.mean(preds == Y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))



Epoch 1/4000
9/9 - 1s - loss: 1.3866 - accuracy: 0.2292 - val_loss: 1.3866 - val_accuracy: 0.2222 - 907ms/epoch - 101ms/step
Epoch 2/4000
9/9 - 0s - loss: 1.3868 - accuracy: 0.2292 - val_loss: 1.3866 - val_accuracy: 0.2222 - 114ms/epoch - 13ms/step
Epoch 3/4000
9/9 - 0s - loss: 1.3864 - accuracy: 0.2500 - val_loss: 1.3865 - val_accuracy: 0.2222 - 91ms/epoch - 10ms/step
Epoch 4/4000
9/9 - 0s - loss: 1.3856 - accuracy: 0.3194 - val_loss: 1.3865 - val_accuracy: 0.2222 - 97ms/epoch - 11ms/step
Epoch 5/4000
9/9 - 0s - loss: 1.3858 - accuracy: 0.2569 - val_loss: 1.3864 - val_accuracy: 0.2222 - 78ms/epoch - 9ms/step
Epoch 6/4000
9/9 - 0s - loss: 1.3857 - accuracy: 0.3194 - val_loss: 1.3863 - val_accuracy: 0.2222 - 81ms/epoch - 9ms/step
Epoch 7/4000
9/9 - 0s - loss: 1.3860 - accuracy: 0.2500 - val_loss: 1.3862 - val_accuracy: 0.2917 - 88ms/epoch - 10ms/step
Epoch 8/4000
9/9 - 0s - loss: 1.3854 - accuracy: 0.2708 - val_loss: 1.3863 - val_accuracy: 0.2917 - 74ms/epoch - 8ms/step
Epoch 9/4000
9/9

Epoch 68/4000
9/9 - 0s - loss: 0.9059 - accuracy: 0.7153 - val_loss: 1.5635 - val_accuracy: 0.2917 - 82ms/epoch - 9ms/step
Epoch 69/4000
9/9 - 0s - loss: 0.8929 - accuracy: 0.7153 - val_loss: 1.6167 - val_accuracy: 0.2917 - 90ms/epoch - 10ms/step
Epoch 70/4000
9/9 - 0s - loss: 0.9097 - accuracy: 0.7014 - val_loss: 1.6261 - val_accuracy: 0.2917 - 77ms/epoch - 9ms/step
Epoch 71/4000
9/9 - 0s - loss: 0.9060 - accuracy: 0.6944 - val_loss: 1.6092 - val_accuracy: 0.2917 - 85ms/epoch - 9ms/step
Epoch 72/4000
9/9 - 0s - loss: 0.8946 - accuracy: 0.7292 - val_loss: 1.5988 - val_accuracy: 0.2917 - 86ms/epoch - 10ms/step
Epoch 73/4000
9/9 - 0s - loss: 0.9029 - accuracy: 0.7361 - val_loss: 1.6508 - val_accuracy: 0.2917 - 84ms/epoch - 9ms/step
Epoch 74/4000
9/9 - 0s - loss: 0.8880 - accuracy: 0.7292 - val_loss: 1.5998 - val_accuracy: 0.2917 - 89ms/epoch - 10ms/step
Epoch 75/4000
9/9 - 0s - loss: 0.8954 - accuracy: 0.7153 - val_loss: 1.5181 - val_accuracy: 0.2917 - 83ms/epoch - 9ms/step
Epoch 76/4000

Epoch 134/4000
9/9 - 0s - loss: 0.7674 - accuracy: 0.7708 - val_loss: 3.1341 - val_accuracy: 0.2222 - 86ms/epoch - 10ms/step
Epoch 135/4000
9/9 - 0s - loss: 0.7641 - accuracy: 0.7917 - val_loss: 3.6623 - val_accuracy: 0.2222 - 78ms/epoch - 9ms/step
Epoch 136/4000
9/9 - 0s - loss: 0.7795 - accuracy: 0.7847 - val_loss: 4.9618 - val_accuracy: 0.2222 - 87ms/epoch - 10ms/step
Epoch 137/4000
9/9 - 0s - loss: 0.7625 - accuracy: 0.7500 - val_loss: 5.6511 - val_accuracy: 0.2222 - 85ms/epoch - 9ms/step
Epoch 138/4000
9/9 - 0s - loss: 0.7663 - accuracy: 0.7986 - val_loss: 6.2759 - val_accuracy: 0.2222 - 79ms/epoch - 9ms/step
Epoch 139/4000
9/9 - 0s - loss: 0.7762 - accuracy: 0.7778 - val_loss: 6.9772 - val_accuracy: 0.2222 - 87ms/epoch - 10ms/step
Epoch 140/4000
9/9 - 0s - loss: 0.7844 - accuracy: 0.7639 - val_loss: 4.8650 - val_accuracy: 0.2222 - 91ms/epoch - 10ms/step
Epoch 141/4000
9/9 - 0s - loss: 0.7737 - accuracy: 0.7639 - val_loss: 4.8212 - val_accuracy: 0.2222 - 82ms/epoch - 9ms/step
Epoc

Epoch 200/4000
9/9 - 0s - loss: 0.6827 - accuracy: 0.7986 - val_loss: 3.9243 - val_accuracy: 0.2500 - 81ms/epoch - 9ms/step
Epoch 201/4000
9/9 - 0s - loss: 0.6726 - accuracy: 0.8056 - val_loss: 3.6664 - val_accuracy: 0.2500 - 96ms/epoch - 11ms/step
Epoch 202/4000
9/9 - 0s - loss: 0.6783 - accuracy: 0.8194 - val_loss: 1.3408 - val_accuracy: 0.2361 - 90ms/epoch - 10ms/step
Epoch 203/4000
9/9 - 0s - loss: 0.6877 - accuracy: 0.8194 - val_loss: 4.2601 - val_accuracy: 0.2361 - 83ms/epoch - 9ms/step
Epoch 204/4000
9/9 - 0s - loss: 0.6847 - accuracy: 0.8056 - val_loss: 1.0708 - val_accuracy: 0.4028 - 92ms/epoch - 10ms/step
Epoch 205/4000
9/9 - 0s - loss: 0.6944 - accuracy: 0.8125 - val_loss: 2.0849 - val_accuracy: 0.2361 - 82ms/epoch - 9ms/step
Epoch 206/4000
9/9 - 0s - loss: 0.6945 - accuracy: 0.8056 - val_loss: 1.1732 - val_accuracy: 0.4306 - 79ms/epoch - 9ms/step
Epoch 207/4000
9/9 - 0s - loss: 0.6646 - accuracy: 0.8403 - val_loss: 4.5722 - val_accuracy: 0.2639 - 80ms/epoch - 9ms/step
Epoch

Epoch 266/4000
9/9 - 0s - loss: 0.6208 - accuracy: 0.8542 - val_loss: 1.1785 - val_accuracy: 0.3472 - 82ms/epoch - 9ms/step
Epoch 267/4000
9/9 - 0s - loss: 0.5936 - accuracy: 0.8819 - val_loss: 7.3868 - val_accuracy: 0.2639 - 88ms/epoch - 10ms/step
Epoch 268/4000
9/9 - 0s - loss: 0.6141 - accuracy: 0.8333 - val_loss: 1.0639 - val_accuracy: 0.4444 - 85ms/epoch - 9ms/step
Epoch 269/4000
9/9 - 0s - loss: 0.6360 - accuracy: 0.8264 - val_loss: 3.1689 - val_accuracy: 0.2917 - 92ms/epoch - 10ms/step
Epoch 270/4000
9/9 - 0s - loss: 0.6200 - accuracy: 0.8472 - val_loss: 3.6767 - val_accuracy: 0.3056 - 90ms/epoch - 10ms/step
Epoch 271/4000
9/9 - 0s - loss: 0.6275 - accuracy: 0.8472 - val_loss: 3.4423 - val_accuracy: 0.2500 - 81ms/epoch - 9ms/step
Epoch 272/4000
9/9 - 0s - loss: 0.6147 - accuracy: 0.8681 - val_loss: 2.6546 - val_accuracy: 0.3056 - 80ms/epoch - 9ms/step
Epoch 273/4000
9/9 - 0s - loss: 0.6317 - accuracy: 0.8194 - val_loss: 1.2823 - val_accuracy: 0.3889 - 84ms/epoch - 9ms/step
Epoch

...

Epoch 2485/4000
9/9 - 0s - loss: 0.0671 - accuracy: 1.0000 - val_loss: 173.2761 - val_accuracy: 0.2222 - 83ms/epoch - 9ms/step
Epoch 2486/4000
9/9 - 0s - loss: 0.0873 - accuracy: 0.9861 - val_loss: 196.3105 - val_accuracy: 0.2222 - 91ms/epoch - 10ms/step
Epoch 2487/4000
9/9 - 0s - loss: 0.0738 - accuracy: 1.0000 - val_loss: 116.6993 - val_accuracy: 0.2222 - 97ms/epoch - 11ms/step
Epoch 2488/4000
9/9 - 0s - loss: 0.0542 - accuracy: 1.0000 - val_loss: 99.3940 - val_accuracy: 0.2222 - 98ms/epoch - 11ms/step
Epoch 2489/4000
9/9 - 0s - loss: 0.0609 - accuracy: 0.9931 - val_loss: 106.4081 - val_accuracy: 0.2222 - 89ms/epoch - 10ms/step
Epoch 2490/4000
9/9 - 0s - loss: 0.0672 - accuracy: 1.0000 - val_loss: 130.5763 - val_accuracy: 0.2222 - 94ms/epoch - 10ms/step
Epoch 2491/4000
9/9 - 0s - loss: 0.0730 - accuracy: 1.0000 - val_loss: 115.0501 - val_accuracy: 0.2222 - 82ms/epoch - 9ms/step
Epoch 2492/4000
9/9 - 0s - loss: 0.0722 - accuracy: 1.0000 - val_loss: 104.8087 - val_accuracy: 0.2917 - 84

Epoch 2550/4000
9/9 - 0s - loss: 0.0549 - accuracy: 1.0000 - val_loss: 7.5196 - val_accuracy: 0.2917 - 97ms/epoch - 11ms/step
Epoch 2551/4000
9/9 - 0s - loss: 0.0733 - accuracy: 1.0000 - val_loss: 36.8639 - val_accuracy: 0.3056 - 93ms/epoch - 10ms/step
Epoch 2552/4000
9/9 - 0s - loss: 0.0736 - accuracy: 0.9931 - val_loss: 118.9725 - val_accuracy: 0.2917 - 92ms/epoch - 10ms/step
Epoch 2553/4000
9/9 - 0s - loss: 0.0805 - accuracy: 0.9722 - val_loss: 78.3328 - val_accuracy: 0.2222 - 81ms/epoch - 9ms/step
Epoch 2554/4000
9/9 - 0s - loss: 0.0564 - accuracy: 1.0000 - val_loss: 87.7464 - val_accuracy: 0.2222 - 88ms/epoch - 10ms/step
Epoch 2555/4000
9/9 - 0s - loss: 0.0679 - accuracy: 0.9931 - val_loss: 84.9136 - val_accuracy: 0.2222 - 91ms/epoch - 10ms/step
Epoch 2556/4000
9/9 - 0s - loss: 0.0580 - accuracy: 1.0000 - val_loss: 29.4705 - val_accuracy: 0.2222 - 83ms/epoch - 9ms/step
Epoch 2557/4000
9/9 - 0s - loss: 0.0681 - accuracy: 1.0000 - val_loss: 22.0194 - val_accuracy: 0.2222 - 83ms/epoc

Epoch 2615/4000
9/9 - 0s - loss: 0.0729 - accuracy: 1.0000 - val_loss: 129.0246 - val_accuracy: 0.2917 - 84ms/epoch - 9ms/step
Epoch 2616/4000
9/9 - 0s - loss: 0.0904 - accuracy: 0.9722 - val_loss: 60.6880 - val_accuracy: 0.2917 - 77ms/epoch - 9ms/step
Epoch 2617/4000
9/9 - 0s - loss: 0.0677 - accuracy: 1.0000 - val_loss: 54.9987 - val_accuracy: 0.2222 - 86ms/epoch - 10ms/step
Epoch 2618/4000
9/9 - 0s - loss: 0.0731 - accuracy: 0.9931 - val_loss: 96.8542 - val_accuracy: 0.2222 - 80ms/epoch - 9ms/step
Epoch 2619/4000
9/9 - 0s - loss: 0.0835 - accuracy: 0.9931 - val_loss: 135.2648 - val_accuracy: 0.2222 - 89ms/epoch - 10ms/step
Epoch 2620/4000
9/9 - 0s - loss: 0.0740 - accuracy: 1.0000 - val_loss: 44.1985 - val_accuracy: 0.2917 - 84ms/epoch - 9ms/step
Epoch 2621/4000
9/9 - 0s - loss: 0.0636 - accuracy: 1.0000 - val_loss: 132.4061 - val_accuracy: 0.2917 - 79ms/epoch - 9ms/step
Epoch 2622/4000
9/9 - 0s - loss: 0.0809 - accuracy: 0.9861 - val_loss: 135.0121 - val_accuracy: 0.2917 - 86ms/epo

Epoch 2680/4000
9/9 - 0s - loss: 0.0794 - accuracy: 0.9931 - val_loss: 82.2078 - val_accuracy: 0.2917 - 86ms/epoch - 10ms/step
Epoch 2681/4000
9/9 - 0s - loss: 0.0711 - accuracy: 0.9861 - val_loss: 9.8364 - val_accuracy: 0.2500 - 93ms/epoch - 10ms/step
Epoch 2682/4000
9/9 - 0s - loss: 0.0553 - accuracy: 1.0000 - val_loss: 39.3994 - val_accuracy: 0.2222 - 86ms/epoch - 10ms/step
Epoch 2683/4000
9/9 - 0s - loss: 0.0587 - accuracy: 1.0000 - val_loss: 29.9835 - val_accuracy: 0.2361 - 96ms/epoch - 11ms/step
Epoch 2684/4000
9/9 - 0s - loss: 0.0585 - accuracy: 1.0000 - val_loss: 65.0519 - val_accuracy: 0.2917 - 96ms/epoch - 11ms/step
Epoch 2685/4000
9/9 - 0s - loss: 0.0641 - accuracy: 1.0000 - val_loss: 21.4989 - val_accuracy: 0.2222 - 88ms/epoch - 10ms/step
Epoch 2686/4000
9/9 - 0s - loss: 0.0633 - accuracy: 0.9931 - val_loss: 10.2289 - val_accuracy: 0.2917 - 90ms/epoch - 10ms/step
Epoch 2687/4000
9/9 - 0s - loss: 0.0494 - accuracy: 1.0000 - val_loss: 41.3451 - val_accuracy: 0.2917 - 94ms/epo

Epoch 2745/4000
9/9 - 0s - loss: 0.0711 - accuracy: 1.0000 - val_loss: 208.9599 - val_accuracy: 0.2222 - 135ms/epoch - 15ms/step
Epoch 2746/4000
9/9 - 3s - loss: 0.0519 - accuracy: 1.0000 - val_loss: 199.6234 - val_accuracy: 0.2222 - 3s/epoch - 278ms/step
Epoch 2747/4000
9/9 - 0s - loss: 0.0665 - accuracy: 1.0000 - val_loss: 97.0499 - val_accuracy: 0.2222 - 118ms/epoch - 13ms/step
Epoch 2748/4000
9/9 - 0s - loss: 0.0896 - accuracy: 0.9861 - val_loss: 114.0482 - val_accuracy: 0.2917 - 101ms/epoch - 11ms/step
Epoch 2749/4000
9/9 - 0s - loss: 0.0886 - accuracy: 0.9861 - val_loss: 155.6877 - val_accuracy: 0.2917 - 119ms/epoch - 13ms/step
Epoch 2750/4000
9/9 - 0s - loss: 0.0904 - accuracy: 0.9861 - val_loss: 4.6742 - val_accuracy: 0.2917 - 100ms/epoch - 11ms/step
Epoch 2751/4000
9/9 - 0s - loss: 0.0704 - accuracy: 1.0000 - val_loss: 26.8991 - val_accuracy: 0.2222 - 104ms/epoch - 12ms/step
Epoch 2752/4000
9/9 - 0s - loss: 0.0787 - accuracy: 0.9931 - val_loss: 124.3551 - val_accuracy: 0.2222 

In [38]:
from mne.decoding import CSP # Common Spatial Pattern Filtering
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers
from sklearn.model_selection import ShuffleSplit

# OneHotEncoding Labels
enc = OneHotEncoder()
X_out = enc.fit_transform(labels.reshape(-1,1)).toarray()

# Cross Validation Split
cv = ShuffleSplit(n_splits = 10, test_size = 0.2, random_state = 0)

from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

acc = []
ka = []
prec = []
recall = []

Computing rank from data with rank=None
    Using tolerance 1.1e-05 (2.2e-16 eps * 22 dim * 2.3e+09  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.1e-05 (2.2e-16 eps * 22 dim * 2.2e+09  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.2e-05 (2.2e-16 eps * 22 dim * 2.5e+09  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.1e-05 (2.2e-16 eps * 22 dim * 2.3e+09  max singular value)


Computing rank from data with rank=None
    Using tolerance 8.7e-07 (2.2e-16 eps * 22 dim * 1.8e+08  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 7e-07 (2.2e-16 eps * 22 dim * 1.4e+08  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 8.1e-07 (2.2e-16 eps * 22 dim * 1.7e+08  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 7e-07 (2.2e-16 eps * 22 dim * 1.4e+08  max singular value)
    

ValueError: in user code:

    File "C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\keras\engine\input_

In [8]:
raw.annotations

<Annotations | 595 segments: 1023 (7), 1072 (1), 276 (1), 277 (1), 32766 ...>

In [10]:
events, event_ids = mne.events_from_annotations(raw)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '783']


In [11]:
stims = [value for key, value in event_ids.items() if key == "783"]

In [16]:
epochs = mne.Epochs(raw, events, event_id=stims, tmin=0, tmax=4,
                            baseline=None, preload=True, proj=False)

Not setting metadata
Not setting metadata
288 matching events found
No baseline correction applied
Loading data for 288 events and 1001 original time points ...
0 bad epochs dropped


In [11]:
"log %s" %('this')

'log this'

(288, 25, 1001)

In [ ]:
raw.info['bads'] += ['EOG-left', 'EOG-central', 'EOG-right']
